In [1]:
import torch
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from ca_utils import ResNet, BasicBlock

# Load the trained model
model = ResNet(block=BasicBlock, layers=[1, 1, 1], num_classes=10)
model.load_state_dict(torch.load("data/weights_resnet(q4).pth"))
model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# Prepare the test data loader
test_transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_data = datasets.ImageFolder(r"C:\Users\risha\Downloads\val", transform=test_transform)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4,pin_memory=True)

# Define the test function
def test_cnn(model, test_loader):
    model.eval()
    device = next(model.parameters()).device
    all_predictions = []
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    return np.array(all_predictions), accuracy

# # Test the model and get predictions and accuracy
# predictions, accuracy = test_cnn(model, test_loader)
# print(f'Classification Accuracy: {accuracy:.2f}%')

# Call the function to get predictions and accuracy
predictions, accuracy = test_cnn(model, test_loader)
print(f'Classification Accuracy: {accuracy:.2f}%')
# Ensure that the predictions are printed out
print(f'Predicted labels: {predictions}')

Classification Accuracy: 45.35%
Predicted labels: [0 0 8 ... 9 9 9]
